In [1]:
##create ouryelpcheckins.csv (DF) from checkins.json (reduce by ourbusinesses.csv)
##
##Input:  yelp file checkins.json
##output: ouryelpcheckins.csv
##filter: filtered list of businesses in ouryelpbusinesses    

## Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import types
import re
#import ijson

import os
from datetime import datetime

%matplotlib inline
##

datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2019-05-28 12:20:37'

In [2]:
# !pip install pyspark

In [3]:
# !pip install ijson

In [4]:
#setup environment
import os
cwd = os.getcwd()
print("Current Working Directory " , cwd)
os.chdir("C:/Users/Richpat/Documents/@syracuse coursework/@IST 718 Big Data Analytics (Python)/finalproject")
print('\nFiles in directory', os.listdir())
print('\n')
pd.show_versions(as_json=True) # True to shorten/flatten output
print ('\nStart date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Current Working Directory  c:\Users\Richpat\Documents\@syracuse coursework\@IST 718 Big Data Analytics (Python)\finalproject

Files in directory ['.ipynb_checkpoints', '16zpallnoagi.csv', '16zpdoc.doc', '2016_Historical_Business_File_Layout.xlsx', 'aclImdb_v1', 'aclImdb_v1.tar', 'aclImdb_v1.tar.gz', 'b.2.1_41473_project_checkpoint_1_clean.pdf', 'b.4.1_41474_project_checkpoint_2_clean.pdf', 'bCatTall.csv', 'business.json', 'BusinessCategoryCount.csv', 'checkin.json', 'Dataset_Challenge_Dataset_Agreement.pdf', 'DBHArizona164RestaurantsHighRisk.csv', 'DBHArizona164RestaurantsLowGT9.csv', 'DBHArizona164RestaurantsLowLT10.csv', 'DBHArizona164RestaurantsMediumGT6.csv', 'DBHArizona164RestaurantsMediumLT7.csv', 'DBHArizona164RestaurantsNARisk.csv', 'DBHNevada164RestaurantsHighRisk.csv', 'DBHNevada164RestaurantsLowRisk.csv', 'DBHNevada164RestaurantsMediumRisk.csv', 'DBHNevada164RestaurantsNARisk.csv', 'DBHRestaurants.csv', 'DBHRestaurantsSource.zip', 'DescribeYelpData.ipynb', 'DRT_IST718_FinalP

In [5]:
def readJSONreturnLIST (fn, verbosity):

    print ('Start readJSONreturnLIST for ', fn, 'at date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    with open(fn, encoding='utf-8') as json_file:      
        data = json_file.readlines()
        # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
        data = list(map(json.loads, data)) 
        
    print ('Type data', type(data), 'len data', len(data))
        
    print ('Start readJSONreturnLIST date/time', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    return (data)

In [6]:
# read in full business.json files (excluding review.json - see later for that)

print ('Start date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
checkins=readJSONreturnLIST ('checkin.json', 2)
print ('\nNumber of elements', len(checkins))
print ('End date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Start date/time: 2019-05-28 12:20:40
Start readJSONreturnLIST for  checkin.json at date/time 2019-05-28 12:20:40
Type data <class 'list'> len data 161950
Start readJSONreturnLIST date/time 2019-05-28 12:20:42

Number of elements 161950
End date/time: 2019-05-28 12:20:42


In [7]:
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
rNocheckinsCnt=0
print ('Sample of checkin data')
print ('=======================')
for checkin in checkins:
    if rNocheckinsCnt==0:
        print (checkin)
        print ('business ID', checkin['business_id'])
        print ('date', checkin['date'])
        print (json.dumps(checkin, indent=4))
    rNocheckinsCnt+=1
print('Number of checkins counted:', rNocheckinsCnt)
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Date/time: 2019-05-28 12:20:42
Sample of checkin data
{'business_id': '--1UhMGODdWsrMastO9DZw', 'date': '2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02'}
business ID --1UhMGODdWsrMastO9DZw
date 2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02
{
    "business_id": "--1UhMGODdWsrMastO9DZw",
    "date": "2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02"
}
Number of checkins counted: 161950
Date/time: 2019-05-28 12:20:42


In [8]:
#convert checkins to DF
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
checkinsDF=pd.DataFrame(checkins)
checkinsDF.index=checkinsDF['business_id']
checkinsDF.drop('business_id', axis=1, inplace=True)  

print("checkinsDF data")
print("===============")

#print("\n***** isna")
#print(checkinsDF.isnull().sum())

print("\n***** dtypes")
print(checkinsDF.dtypes)

print("\n***** head")
print(pd.DataFrame.head(checkinsDF))

print("\n***** describe")
print(checkinsDF.describe())

print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Date/time: 2019-05-28 12:20:42
checkinsDF data

***** dtypes
date    object
dtype: object

***** head
                                                                     date
business_id                                                              
--1UhMGODdWsrMastO9DZw  2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016...
--6MefnULPED_I942VcFNA  2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012...
--7zmmkVg-IMGaXbuVd0SQ  2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015...
--8LPVSo5i0Oo61X01sV9A                                2016-07-08 16:43:30
--9QQLMTbFzLJ_oT-ON3Xw  2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010...

***** describe
                       date
count                161950
unique               161949
top     2015-03-28 22:10:26
freq                      2
Date/time: 2019-05-28 12:20:42


In [9]:
#load "our businesses"
#
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurBusinessesDF = pd.read_csv('ouryelpbusinesses.csv', sep='|', encoding='utf-8')
OurBusinessesDF.index=OurBusinessesDF['business_id']
OurBusinessesDF.drop('business_id', axis=1, inplace=True) 
print ('Date/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

print("OurBusinessesDF data")
print("====================")

print("\n***** head")
print(pd.DataFrame.head(OurBusinessesDF))


Date/time: 2019-05-28 12:20:42
Date/time: 2019-05-28 12:20:43
OurBusinessesDF data

***** head
                        Unnamed: 0                       address  \
business_id                                                        
1Dfx3zM-rW4n-31KeC8sJg          11       2450 E Indian School Rd   
PZ-LZzSlhSe9utkQYU8pFg          17  1775 E Tropicana Ave, Ste 29   
LB6ZyCfUzeX9OLdunHhnOQ          30        8800 W Charleston Blvd   
HVPcIcqiJkrpD36xZFGN6g          34           3150 N Rainbow Blvd   
0_ohldeFwysbglrTLSGM4Q          36              15224 N 59th Ave   

                                                               attributes  \
business_id                                                                 
1Dfx3zM-rW4n-31KeC8sJg  {'RestaurantsTakeOut': 'True', 'BusinessParkin...   
PZ-LZzSlhSe9utkQYU8pFg  {'OutdoorSeating': 'False', 'BusinessAcceptsCr...   
LB6ZyCfUzeX9OLdunHhnOQ  {'BusinessAcceptsCreditCards': 'True', 'Restau...   
HVPcIcqiJkrpD36xZFGN6g  {'BusinessAcceptsCr

In [18]:
#Reduce the User DF to have only those users that are in the OurReviewsDF

print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurCheckinsDF=checkinsDF.join(OurBusinessesDF,on='business_id', how='inner')
OurCheckinsDF=pd.DataFrame(OurCheckinsDF['date'])
print ("\nNumber of businesses on the OurBusinessesDF", len(OurBusinessesDF))
print ("\nNumber of checkins on the checkinsDF", len(checkinsDF))
print ("\nNumber of checkins/businesses on the OurCheckinsDF", len(OurCheckinsDF))

print("OurCheckinsDF data")
print("====================")

#print("\n***** isna")
#print(OurCheckinsDF.isnull().sum())

print("\n***** dtypes")
print(OurCheckinsDF.dtypes)

print("\n***** head")
print(pd.DataFrame.head(OurCheckinsDF))

print("\n***** describe")
print(OurCheckinsDF.describe())

print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


Date/time: 2019-05-28 12:39:04

Number of businesses on the OurBusinessesDF 19122

Number of checkins on the checkinsDF 161950

Number of checkins/businesses on the OurCheckinsDF 16089
OurCheckinsDF data

***** dtypes
date    object
dtype: object

***** head
                                                                     date
business_id                                                              
--9QQLMTbFzLJ_oT-ON3Xw  2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010...
--TcDRzRIxhvHM4DSgEuMA  2012-03-05 04:07:17, 2012-05-23 07:00:13, 2013...
-050d_XIor1NpCuWkbIVaQ  2010-01-23 14:45:07, 2010-01-27 17:59:13, 2010...
-0BxAGlIk5DJAGVkpqBXxg  2010-08-20 19:44:47, 2010-08-26 23:54:03, 2010...
-0t6o9LXMVQ3aV8CHhRTnA  2016-04-20 19:59:51, 2016-04-20 22:42:56, 2016...

***** describe
                                                     date
count                                               16089
unique                                              16089
top     2016-01-03 17:27:45, 201

In [11]:
print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
OurCheckinsDF.to_csv("ouryelpcheckins.csv", sep=",", encoding = 'utf-8')
print ('\nDate/time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


Date/time: 2019-05-28 12:20:43

Date/time: 2019-05-28 12:20:44
